In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Read in the data
train_df = pd.read_csv('of dataset/train.csv')
test_df = pd.read_csv('of dataset/val.csv')
train_df.head()


,question,answer
0,A geologist would be most likely to investigat...,mountains form.
1,Which type of Earth scientist might look for p...,geologist
2,Chemical oceanography is the study of the,naturally occurring elements in ocean water.
3,The problem of global warming is most likely t...,climatologist.
4,Which type of Earth scientist would you expect...,meteorologist


In [52]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

#tdfidvectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#embeddings from tensorflow hub
import tensorflow_hub as hub
embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"


In [53]:
embedding

'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1'

In [54]:

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_df['question'])
train_df['question'] = tokenizer.texts_to_sequences(train_df['question'])
tokenizer.fit_on_texts(train_df['answer'])
train_df['answer'] = tokenizer.texts_to_sequences(train_df['answer'])

tokenizer.fit_on_texts(test_df['question'])
test_df['question'] = tokenizer.texts_to_sequences(test_df['question'])
tokenizer.fit_on_texts(test_df['answer'])
test_df['answer'] = tokenizer.texts_to_sequences(test_df['answer'])

train_df.head()



,question,answer
0,"[3, 3751, 150, 24, 29, 331, 6, 2831, 41]","[728, 62]"
1,"[10, 47, 2, 32, 402, 1103, 955, 17, 690, 17, 1...",[3318]
2,"[51, 2285, 4, 1, 589, 2, 1]","[1170, 2156, 100, 5, 65, 15]"
3,"[1, 562, 2, 474, 691, 4, 29, 331, 6, 24, 1, 83...",[5968]
4,"[10, 47, 2, 32, 402, 150, 31, 1494, 6, 956, 3,...",[5969]


In [55]:
max_length_question = max([len(s) for s in train_df['question']])
max_length_answer = max([len(s) for s in train_df['answer']])
print('Max length question: %d' % max_length_question)
print('Max length answer: %d' % max_length_answer)

#Pad the questions and answers
train_df['question'] = tf.keras.preprocessing.sequence.pad_sequences(train_df['question'], maxlen=max_length_question, padding='post')
train_df['answer'] = tf.keras.preprocessing.sequence.pad_sequences(train_df['answer'], maxlen=max_length_answer, padding='post')

test_df['question'] = tf.keras.preprocessing.sequence.pad_sequences(test_df['question'], maxlen=max_length_question, padding='post')
test_df['answer'] = tf.keras.preprocessing.sequence.pad_sequences(test_df['answer'], maxlen=max_length_answer, padding='post')


Max length question: 57
Max length answer: 20


In [56]:
import numpy as np
for i in range(len(train_df['question'])):
    train_df['question'][i] = np.asarray(train_df['question'][i]).astype('float32')
for i in range(len(train_df['answer'])):
    train_df['answer'][i] = np.asarray(train_df['answer'][i]).astype('float32')

for i in range(len(test_df['question'])):
    test_df['question'][i] = np.asarray(test_df['question'][i]).astype('float32')
for i in range(len(test_df['answer'])):
    test_df['answer'][i] = np.asarray(test_df['answer'][i]).astype('float32')

In [61]:
#tensorflow model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers  import Embedding, LSTM, Dense, Bidirectional, Dropout, Input
from tensorflow.keras import Sequential

#bert
import tensorflow_hub as hub

#bert
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=True)


model = Sequential()
model.add(bert_layer)
model.add(Dense(1, activation='sigmoid'))


model.compile(
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'],
                optimizer = tf.keras.optimizers.Adam(1e-4)
            )

model.summary()


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [59]:
history = model.fit(
    train_df['question'], train_df['answer'], epochs=10, batch_size=32, validation_data=(test_df['question'], test_df['answer']))

Epoch 1/10
248/248 [==============================] - 13s 32ms/step - loss: -94.8813 - accuracy: 1.2621e-04 - val_loss: -290.2776 - val_accuracy: 0.0000e+00
Epoch 2/10
248/248 [==============================] - 8s 32ms/step - loss: -1999.6578 - accuracy: 0.0289 - val_loss: -4415.2803 - val_accuracy: 0.0368
Epoch 3/10
248/248 [==============================] - 7s 29ms/step - loss: -12532.5732 - accuracy: 0.0327 - val_loss: -16513.2402 - val_accuracy: 0.0368
Epoch 4/10
248/248 [==============================] - 7s 29ms/step - loss: -27336.0117 - accuracy: 0.0327 - val_loss: -32353.4551 - val_accuracy: 0.0368
Epoch 5/10
248/248 [==============================] - 8s 31ms/step - loss: -43280.5898 - accuracy: 0.0327 - val_loss: -49795.5703 - val_accuracy: 0.0368
Epoch 6/10
248/248 [==============================] - 10s 39ms/step - loss: -60548.6875 - accuracy: 0.0327 - val_loss: -68037.5781 - val_accuracy: 0.0368
Epoch 7/10
248/248 [==============================] - 8s 33ms/step - loss: -780

In [29]:
#scikit learn model sequence to sequence model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split





ValueError: setting an array element with a sequence.